In [8]:
%run anomaly_detection_main.ipynb

1. Add different data set
4. Order book data change to sliding window

In [43]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.neighbors import LocalOutlierFactor

In [10]:
# def geobrownian_contam_sample(normal_var,abnormal_var,n,nPath,normal_ratio):
    
#     mu = 0
#     S0 = 1

#     t = np.linspace(0, 1, n)
#     dt = t[1] - t[0]
#     nPath_normal = np.int(nPath*normal_ratio) 

#     W_n = np.random.normal(0,normal_var*dt,(nPath_normal,n)) 
#     W_a = np.random.normal(0,abnormal_var*dt,((nPath - nPath_normal),n))

#     W = np.cumsum(np.concatenate((W_n,W_a))) ### standard brownian motion ###

#     sigma = np.concatenate((np.ones(nPath_normal)*normal_var,np.ones(nPath - nPath_normal)*abnormal_var))
#     X = -0.5*np.outer(sigma,t) 
#     Y = W*np.sqrt(sigma)
#     S = np.exp(X) ### geometric brownian motion ###

#     return ,t,nPath_normal

In [11]:
# n,nPath_train,nPath_test,normal_ratio=100,1000,500,0.95
# # the drifts for define normal and abnormal path
# # leadlag_transform
# normal_var,abnormal_var = 1,1.5

# geobrown_train_var,geo_time_train_var,geo_nPath_train_normal_var = geobrownian_contam_sample(normal_var,abnormal_var,n,nPath_train,normal_ratio)
# geoown_test_var,geo_time_test_var,geo_nPath_test_normal_var = geobrownian_contam_sample(normal_var,abnormal_var,n,nPath_test,normal_ratio)

### Simulated Brownian Motion Data Contaminated with Different Variance

In [12]:
def brownian_contamvar_sample(normal_var,abnormal_var,n,nPath,normal_ratio): 
    
    tt=np.linspace(0,1,n)
    dt=tt[1]-tt[0]
    
    nPath_normal = np.int(nPath*normal_ratio) 
    
    #construct normal and anomaly path

    nn = np.random.normal(0,normal_var*dt,(nPath_normal,n))
    aa = np.random.normal(0,abnormal_var*dt,((nPath - nPath_normal),n))
    zz = np.concatenate((nn,aa))
    ww = np.cumsum(zz,axis=1)
    
    label = np.concatenate((np.zeros(np.int(nPath*normal_ratio)),np.ones(np.int(nPath*(1-normal_ratio)))))
    ww_shuffled,label_shuffled = shuffle(ww, label, random_state=0)
    
    return ww_shuffled,label_shuffled

In [93]:
n,nPath,normal_ratio=10,800,0.95
# var for normal and abnormal path
normal_var,abnormal_var = 1,3

brown_contam,brown_contam_label = brownian_contamvar_sample(normal_var,abnormal_var,n,nPath,normal_ratio)

#### Isolation Forest

In [94]:
kf = KFold(n_splits=5,shuffle=True)
fbeta_iso = []
for train_index, test_index in kf.split(brown_contam):
    brownpath_train, brownpath_test = brown_contam[train_index], brown_contam[test_index]
    brownlabel_train, brownlabel_test = brown_contam_label[train_index], brown_contam_label[test_index]
    
    viforest = IsolationForest(n_estimators = 100,max_samples='auto', contamination=float(.1)).fit(brownpath_train)
    vif_pred_label_test = viforest.predict(brownpath_test)
    
    where_0 = np.where(brownlabel_test == 0)
    where_1 = np.where(brownlabel_test == 1)
    brownlabel_test[where_0] = 1
    brownlabel_test[where_1] = -1
    
    fbeta_iso.append(fbeta_score(brownlabel_test,vif_pred_label_test,average='macro', beta=2))

In [95]:
fbeta_iso_mean = np.mean(np.array(fbeta_iso))
fbeta_iso_mean

0.8641203267646425

#### LOF

In [110]:
kf = KFold(n_splits=5,shuffle=True)
fbeta_lof = []
for train_index, test_index in kf.split(brown_contam):
    brownpath_train, brownpath_test = brown_contam[train_index], brown_contam[test_index]
    brownlabel_train, brownlabel_test = brown_contam_label[train_index], brown_contam_label[test_index]
    clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1, novelty=True)
    clf.fit(brownpath_train)
    lof_y_pred = clf.predict(brownpath_test)
    
    where_0 = np.where(brownlabel_test == 0)
    where_1 = np.where(brownlabel_test == 1)
    brownlabel_test[where_1] = -1
    brownlabel_test[where_0] = 1
    
    fbeta_lof.append(fbeta_score(brownlabel_test,lof_y_pred,average='macro', beta=2))
fbeta_lof_mean = np.mean(np.array(fbeta_lof))
fbeta_lof_mean

0.889041510841406

In [108]:
fbeta_lof_mean

[0.8627843660828818,
 0.9660516776652106,
 0.9052309782608696,
 0.8870567815521944,
 0.8522591155191415]

#### KNN

In [86]:
kf = KFold(n_splits=5,shuffle=True)
fbeta_knn = []
for train_index, test_index in kf.split(brown_contam):
    brownpath_train, brownpath_test = brown_contam[train_index], brown_contam[test_index]
    brownlabel_train, brownlabel_test = brown_contam_label[train_index], brown_contam_label[test_index]
    
    knn = KNeighborsClassifier(n_neighbors=10)
    knn.fit(brownpath_train, brownlabel_train)
    y_pred = knn.predict(brownpath_test)
    
    fbeta_knn.append(fbeta_score(brownlabel_test,y_pred,average='macro', beta=2))

In [87]:
fbeta_knn_mean = np.mean(np.array(fbeta_knn))
fbeta_knn_mean

0.6769865925509564

#### Conformance Anomaly Detection

In [37]:
# Transform data
def geobrow_transfrom_paths(ww,nPath,normal_ratio,addtime, leadlag):
    
    ll = transform(ww,at = addtime,ll = leadlag)
     
    #label 0:normal 1:anomaly
    nPath_normal = np.int(nPath*normal_ratio)
    label = np.concatenate((np.zeros(nPath_normal),np.ones(nPath - nPath_normal)))

    ll_shuffled,y_ll_shuffled = shuffle(ll, label, random_state=0)
    
    return np.array(ll_shuffled),y_ll_shuffled

In [38]:
def pred_result(x,y,x_test,y_test):
    #sig_order,epsilon = 3, 0.9
    X1,X2,X1_label,X_test = sign_splitdata(x,y,x_test,3)
    thershold_ratio,X1_pred_label= thershold(X1,X2,0.9)
    pred_conf,pred_test_label = pred_anomaly(X_test,X2,thershold_ratio,y_test)
    
    return X1_label,X1_pred_label,thershold_ratio,pred_conf,pred_test_label

In [39]:
vat_brownpath = transform(brown_contam,at = False,ll = True)

In [101]:
kf = KFold(n_splits=5,shuffle=True)
fbeta_conf_anomaly = []
for train_index, test_index in kf.split(vat_brownpath):
    vat_brownpath_train, vat_brownpath_test = vat_brownpath[train_index], vat_brownpath[test_index]
    vat_brownlabel_train, vat_brownlabel_test = brown_contam_label[train_index], brown_contam_label[test_index]
    X1_label_browian,X1_pred_label_browian,conf_threshold,pred_conf_browian,pred_label_browian = pred_result(vat_brownpath_train,vat_brownlabel_train,vat_brownpath_test,vat_brownlabel_test)
    fbeta_conf_anomaly.append(fbeta_score(vat_brownlabel_test,pred_label_browian,average='macro', beta=2))
fbeta_conf_anomaly_mean = np.mean(np.array(fbeta_conf_anomaly))

In [102]:
fbeta_conf_anomaly

[0.9345815124559107,
 0.9031092925159758,
 0.9439634439634439,
 0.8038720538720538,
 0.8486842105263157]

In [103]:
fbeta_conf_anomaly_mean = np.mean(np.array(fbeta_conf_anomaly))
fbeta_conf_anomaly_mean

0.88684210266674

### Yahoo data

In [ ]:
file_num = 2
yahoo_read_df = pd.read_csv('data_yahoo/ydata_v1_0/A1Benchmark/real_{}.csv'.format(file_num))

In [ ]:
yahoo_read_df.head()

In [ ]:
kf = KFold(n_splits=5,shuffle=True)
Yahoo_fbeta_knn = []
X = yahoo_read_df['value'].values.reshape(-1, 1)
Y= yahoo_read_df['is_anomaly'].values.reshape(-1, 1)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train, Y_train)
    y_pred = knn.predict(X_test)
    
    Yahoo_fbeta_knn.append(fbeta_score(Y_test,y_pred,average='macro', beta=2))
    
Yahoo_fbeta_knn_mean = np.mean(np.array(Yahoo_fbeta_knn))

In [ ]:
Yahoo_fbeta_knn_mean

In [ ]:
X_transform = transform(yahoo_read_df[['timestamp','value']].values.reshape(2, -1),at = False,ll = True)[0].reshape((-1,1,2))

In [ ]:
kf = KFold(n_splits=5,shuffle=True)
Yahoo_fbeta_conf_anomaly = []
for train_index, test_index in kf.split(X_leadlag):
    vat_X_train, vat_X_test = X_leadlag[train_index], X_leadlag[test_index]
    vat_Y_train, vat_Y_test = Y_transform[train_index], Y_transform[test_index]
    X1_label_browian,X1_pred_label_browian,conf_threshold,pred_conf_browian,pred_label_browian = pred_result(vat_X_train,vat_Y_train,vat_X_test,vat_Y_test)
    Yahoo_fbeta_conf_anomaly.append(fbeta_score(vat_Y_test,pred_label_browian,average='macro', beta=2))

In [ ]:
Yahoo_fbeta_conf_anomaly_mean = np.mean(np.array(Yahoo_fbeta_conf_anomaly))
Yahoo_fbeta_conf_anomaly_mean

### No use

In [82]:
viforest = IsolationForest(n_estimators = 100,max_samples='auto', contamination=float(.1)).fit(brown_train)
vif_pred_label_test = viforest.predict(brown_test)

In [83]:
vif_pred_label_test[vif_pred_label_test == 1] = 0
vif_pred_label_test[vif_pred_label_test == -1] = 1

In [72]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(vIso_brown_train, vIso_label_train)
y_pred = knn.predict(vIso_brown_test)
print(fbeta_score(vIso_label_test, y_pred, average='macro', beta=2))

0.4947916666666667


In [84]:
fbeta_score(brown_label_test, vif_pred_label_test, average='macro', beta=2)

0.5819339895773016

In [42]:
# Lead-lag transform and label
vat_brownpath_train,vat_brownlabel_train = geobrow_transfrom_paths(brown_train_var,nPath_train,normal_ratio,False,True)
vat_brownpath_test,vat_brownlabel_test = geobrow_transfrom_paths(brown_test_var,nPath_test,normal_ratio,False,True)estimators =

In [106]:
X1_label_browian,X1_pred_label_browian,conf_threshold,pred_conf_browian,pred_label_browian = pred_result(vat_brownpath_train,vat_brownlabel_train,vat_brownpath_test,vat_brownlabel_test)